In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from WIreless_encoder.TXDataset import TXDataset
from WIreless_encoder.model import WirelessEncoder
import torch

img = cv2.imread('map/map_wireframe.png')

In [2]:
# Define the tokenizer
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')

Using cache found in C:\Users\liuxi/.cache\torch\hub\huggingface_pytorch-transformers_main


In [3]:
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
# Create the dataset and dataloader

# Custom collate function for padding sequences
def custom_collate(batch):
    # Separate the input data (txnames) and labels
    txnames = [item[0] for item in batch]
    labels = torch.tensor([item[1] for item in batch])

    # Pad the sequences in txnames using pad_sequence
    padded_txnames = pad_sequence(txnames, batch_first=True)

    # Return the padded input data and labels
    return padded_txnames, labels

dataset = TXDataset('raw.csv', tokenizer)
dataloader = DataLoader(dataset, batch_size=32, collate_fn=custom_collate)

C:\Users\liuxi\Git Repo\WirelessDT\Source\WiTracingPy\RL\WIreless_encoder\TXDataset.py:16: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  self.data = self.data.groupby(['timestamp', 'x', 'y', 'z'])['rssi', 'tx'].agg(tolist).reset_index()


In [4]:
dataset.data

,timestamp,x,y,z,rssi,tx
0,1679538902693,475.289673,1482.590576,102.184547,"[-58, -70, -67, -66, -62, -58]","[TX_F1_Fountain, TX_F1_Instrument_4, TX_F1_Ins..."
1,1679538902759,476.011902,1483.588623,110.404449,"[-57, -72, -67, -67, -63, -58]","[TX_F1_Fountain, TX_F1_Instrument_4, TX_F1_Ins..."
2,1679538902794,486.526764,1481.530273,110.504608,"[-57, -69, -65, -65, -64, -59]","[TX_F1_Fountain, TX_F1_Instrument_4, TX_F1_Ins..."
3,1679538902828,496.901855,1474.441528,110.863770,"[-57, -71, -66, -67, -63, -60]","[TX_F1_Fountain, TX_F1_Instrument_4, TX_F1_Ins..."
4,1679538902860,506.555115,1461.993652,111.978073,"[-57, -70, -67, -66, -62, -60]","[TX_F1_Fountain, TX_F1_Instrument_4, TX_F1_Ins..."
...,...,...,...,...,...,...
6292,1679539160779,-693.878540,7103.696777,114.846436,"[-74, -73, -61, -65, -55, -58, -64]","[TX_F1_DressCode_17, TX_F1_DressCode_18, TX_F1..."
6293,1679539160804,-693.678711,7106.986816,114.070290,"[-73, -74, -62, -65, -54, -58, -64]","[TX_F1_DressCode_17, TX_F1_DressCode_18, TX_F1..."
6294,1679539160831,-693.429626,7110.733398,113.161743,"[-73, -75, -61, -65, -54, -58, -64]","[TX_F1_DressCode_17, TX_F1_DressCode_18, TX_F1..."
6295,1679539160858,-693.255554,7114.434570,112.097168,"[-73, -74, -62, -65, -54, -58, -64]","[TX_F1_DressCode_17, TX_F1_DressCode_18, TX_F1..."


In [5]:
import torch.nn as nn
import torch.optim as optim

# Create a neural network instance with 1000 possible words in the vocabulary, 16 embedding dimensions, and a hidden size of 64
model = WirelessEncoder(num_words=tokenizer.vocab_size, embedding_dim=16, hidden_size=64).to("cuda")

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100

In [6]:
for epoch in range(num_epochs):
    running_loss = 0.0

    for idx, batch in enumerate(dataloader):
        # print("Batch: ", idx)
        TXName, label = batch
         # Convert the tensors to Float
        TXName = TXName.to("cuda")
        label = label.float().to("cuda")
        # Zero the gradients
        optimizer.zero_grad()

        # Perform feedforward calculations and get the output
        output = model(TXName)

        # Define the target coordinates
        target = label

        # Compute the loss
        loss = criterion(output, target)

        # Perform backpropagation and update the parameters
        loss.backward()
        optimizer.step()

        # Update the running loss
        running_loss += loss.item()

    # Print the average loss for the epoch
    print("Epoch %d, loss: %.3f" % (epoch + 1, running_loss / len(dataset)))

Epoch 1, loss: 44212.467
Epoch 2, loss: 53092.647
Epoch 3, loss: 43576.357
Epoch 4, loss: 53231.727
Epoch 5, loss: 46371.064
Epoch 6, loss: 46942.244
Epoch 7, loss: 47976.798
Epoch 8, loss: 48082.109
Epoch 9, loss: 44200.282
Epoch 10, loss: 39892.449
Epoch 11, loss: 20941.595
Epoch 12, loss: 10889.954
Epoch 13, loss: 9906.260
Epoch 14, loss: 9695.322
Epoch 15, loss: 9972.127
Epoch 16, loss: 9708.783
Epoch 17, loss: 9469.646
Epoch 18, loss: 8894.904
Epoch 19, loss: 8542.601
Epoch 20, loss: 8445.325
Epoch 21, loss: 8444.957
Epoch 22, loss: 8375.381
Epoch 23, loss: 8324.323
Epoch 24, loss: 8268.936
Epoch 25, loss: 8221.993
Epoch 26, loss: 8159.649
Epoch 27, loss: 8108.817
Epoch 28, loss: 8036.282
Epoch 29, loss: 7975.452
Epoch 30, loss: 7892.875
Epoch 31, loss: 7816.675
Epoch 32, loss: 7701.143
Epoch 33, loss: 7601.132
Epoch 34, loss: 7473.079
Epoch 35, loss: 7335.505
Epoch 36, loss: 7119.005
Epoch 37, loss: 7892.846
Epoch 38, loss: 6122.812
Epoch 39, loss: 4175.956
Epoch 40, loss: 3840.0

In [ ]:
with torch.no_grad():
    tx_name_ids = torch.tensor(tokenizer.encode("TX_MainEntrence_Right--------- TX_F1_Fountain---------------- TX_F1_Instrument_4------------ TX_F1_Instrument_5------------ TX_F1_Instrument_6------------ TX_F1_Fountain2--------------- TX_F1_Fountain3--------------- ------------------------------ ------------------------------ ------------------------------", padding='max_length', max_length=1024, add_special_tokens=True)).unsqueeze(0).to("cuda")
    output = model(tx_name_ids)
    print(output)